In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

/tmp/ipykernel_15124/912229180.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
import findspark
findspark.init()

In [3]:
from pyspark.sql import SparkSession

# Initialize Spark session with the legacy time parser policy and other configurations
spark = SparkSession.builder \
    .appName("USPS_scans") \
    .config("spark.executor.memory", "8G") \
    .config("spark.driver.memory", "8G") \
    .config("spark.executor.cores", "4") \
    .config("spark.driver.maxResultSize", "4G") \
    .config("spark.sql.legacy.timeParserPolicy", "LEGACY") \
    .getOrCreate()


24/07/12 10:19:09 WARN Utils: Your hostname, debian resolves to a loopback address: 127.0.1.1; using 192.168.1.20 instead (on interface wlp61s0)
24/07/12 10:19:09 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/07/12 10:19:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/07/12 10:19:10 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
# read csv file with defined schema into Spark DataFrame, and use "," delimiter
# Read all CSV files into a DataFrame
df = spark.read.csv("/home/pk/DAEN690/package_data/PKG_Origin_PcScanLevel.txt", header=True, sep="|")

# Show the DataFrame to verify the changes
df.show(3)

+-----------+---------------+------------+--------------------+------------+--------------------+
|MailPieceID|TrackingEventID|EventZIPCode|   PTSEventTimestamp|PTSEventCode|        PTSEventDesc|
+-----------+---------------+------------+--------------------+------------+--------------------+
|88266058993|        1000032|       21904|1/19/2024 12:26:3...|          VC|PACKAGE RESEARCH ...|
|88251887847|        1000011|       24501|1/20/2024 09:04:1...|          VC|PACKAGE RESEARCH ...|
|88310099929|        1000028|       33605|1/20/2024 10:30:2...|          LX| LOOP MAIL EXCEPTION|
+-----------+---------------+------------+--------------------+------------+--------------------+
only showing top 3 rows



In [5]:
# Convert date fields to datetime format
from pyspark.sql.functions import to_timestamp
date_time_fields = ["PTSEventTimestamp"]

# Convert the PTSEventTimestamp column to datetime

df = df.withColumn("PTSEventTimestamp", to_timestamp("PTSEventTimestamp", "M/d/yyyy HH:mm:ss"))
    
df.printSchema()

root
 |-- MailPieceID: string (nullable = true)
 |-- TrackingEventID: string (nullable = true)
 |-- EventZIPCode: string (nullable = true)
 |-- PTSEventTimestamp: timestamp (nullable = true)
 |-- PTSEventCode: string (nullable = true)
 |-- PTSEventDesc: string (nullable = true)



In [6]:
from pyspark.sql.functions import regexp_extract, col

# Extract numbers and convert to integer
df = df.withColumn("PTSEventCode_Numbers", regexp_extract("PTSEventCode", r'\b\d{1,2}\b', 0).cast("int"))


In [7]:
df.show(3)

+-----------+---------------+------------+-------------------+------------+--------------------+--------------------+
|MailPieceID|TrackingEventID|EventZIPCode|  PTSEventTimestamp|PTSEventCode|        PTSEventDesc|PTSEventCode_Numbers|
+-----------+---------------+------------+-------------------+------------+--------------------+--------------------+
|88266058993|        1000032|       21904|2024-01-19 12:26:35|          VC|PACKAGE RESEARCH ...|                NULL|
|88251887847|        1000011|       24501|2024-01-20 09:04:13|          VC|PACKAGE RESEARCH ...|                NULL|
|88310099929|        1000028|       33605|2024-01-20 10:30:27|          LX| LOOP MAIL EXCEPTION|                NULL|
+-----------+---------------+------------+-------------------+------------+--------------------+--------------------+
only showing top 3 rows



In [32]:
# Register the DataFrame as a temporary view
df.createOrReplaceTempView("scans")

# Run the SQL query to calculate time delta and count scans
result_df = spark.sql("""
    SELECT
    count(*) 
    from
    scans
""")

# Show the result
result_df.show()

+--------+
|count(1)|
+--------+
|43524579|
+--------+



In [20]:
# Register the DataFrame as a temporary view
df.createOrReplaceTempView("scans")

# Run the SQL query to calculate time delta and count scans
result_df = spark.sql("""
    SELECT
    count(distinct PTSEventCode),
    count(distinct PTSEventCode_Numbers)
    FROM
    scans
    --order by PTSEventCode_Numbers
""")

# Show the result
result_df.show()

+----------------------------+------------------------------------+
|count(DISTINCT PTSEventCode)|count(DISTINCT PTSEventCode_Numbers)|
+----------------------------+------------------------------------+
|                         132|                                  69|
+----------------------------+------------------------------------+



In [8]:
# Register the DataFrame as a temporary view
df.createOrReplaceTempView("scans")

# Run the SQL query to calculate time delta and count scans
result_df = spark.sql("""
    SELECT
    distinct PTSEventCode_Numbers,
    PTSEventCode,
    PTSEventDesc
    FROM
    scans
    where PTSEventCode_Numbers is not NULL 
    order by PTSEventCode,PTSEventCode_Numbers
""")

result_df.toPandas().to_csv('PTSEventDesc.csv')
# Show the result
result_df.show()

+--------------------+------------+--------------------+
|PTSEventCode_Numbers|PTSEventCode|        PTSEventDesc|
+--------------------+------------+--------------------+
|                   1|          01|           DELIVERED|
|                   2|          02|         NOTICE LEFT|
|                   3|          03|    ACCEPT OR PICKUP|
|                   4|          04|             REFUSED|
|                   5|          05|UNABLE TO DELIVER...|
|                   6|          06|           FORWARDED|
|                   7|          07|     ARRIVAL AT UNIT|
|                   8|          08|             MISSENT|
|                   9|          09|    RETURN TO SENDER|
|                  10|          10|PROCESSED THROUGH...|
|                  11|          11|SEIZED BY LAW ENF...|
|                  12|          12|      VISIBLE DAMAGE|
|                  13|          13|    AUTHORIZED AGENT|
|                  14|          14|AVAILABLE FOR PICKUP|
|                  15|         

In [9]:
df_temp = result_df.toPandas()

In [23]:
# Register the DataFrame as a temporary view
df.createOrReplaceTempView("scans")

# Run the SQL query to calculate time delta and count scans
result_df = spark.sql("""
    with temp as (
        SELECT
        MailPieceID,
        PTSEventTimestamp,
        PTSEventTimestamp,
        EventZIPCode,
        PTSEventDesc       
        
    FROM scans
    
    where PTSEventCode_Numbers is not NULL 
    and PTSEventCode_Numbers in (7,10)
    
    and trim(eventzipcode) in ('37013','37027','37072','37076','37115','37138','37201','37203','37204','37205',
                         '37206','37207','37208','37209','37210','37211','37212','37214','37215','37216',
                         '37217','37218','37219','37220','37221','37027','37064','37067','37069','37135',
                         '37014','37046','37062','37179','37174','37122','37121','37087','37090','37184',
                         '37138','37075','37075','37066','37148','37070','37072','37048','37188','37086',
                         '37167','37127','37218','37130')
        )
              
        SELECT
        count(*)
        
    FROM temp
    
    
""")

# Show the result
result_df.show()

+--------+
|count(1)|
+--------+
|   49432|
+--------+



In [24]:
# Register the DataFrame as a temporary view
df.createOrReplaceTempView("scans")

# Run the SQL query to calculate time delta and count scans
result_df = spark.sql("""
    with temp as (
        SELECT
        MailPieceID,
        PTSEventTimestamp,
        PTSEventTimestamp,
        EventZIPCode,
        PTSEventDesc       
        
    FROM scans
    
    where PTSEventCode_Numbers is not NULL 
    and PTSEventCode_Numbers in (7,10)
    
    and trim(eventzipcode) in ('37013','37027','37072','37076','37115','37138','37201','37203','37204','37205',
                         '37206','37207','37208','37209','37210','37211','37212','37214','37215','37216',
                         '37217','37218','37219','37220','37221','37027','37064','37067','37069','37135',
                         '37014','37046','37062','37179','37174','37122','37121','37087','37090','37184',
                         '37138','37075','37075','37066','37148','37070','37072','37048','37188','37086',
                         '37167','37127','37218','37130')
        )
              
        SELECT
        MailPieceID,
        MIN(PTSEventTimestamp) AS min_scan_datetime,
        MAX(PTSEventTimestamp) AS max_scan_datetime,
        COUNT (DISTINCT EventZIPCode) AS Distinct_zip_scans,
        count(distinct PTSEventDesc) as Distinct_event_scans,
        (DATEDIFF(MAX(PTSEventTimestamp), MIN(PTSEventTimestamp)) * 24 * 60) AS time_delta_minutes
        
    FROM temp
    group by MailPieceID
    order by Distinct_zip_scans desc
    
""")

# Show the result
# result_df.show()

In [25]:
result_df.count()

44476

In [11]:
# Save the result as a Parquet file
result_df.write.mode("overwrite").parquet("/home/pk/DAEN690/Scans_processed.parquet")

In [33]:
spark.stop()